## Decision Tree implementation from scratch

In [3]:
import numpy as np;
import pandas as pd;
import matplotlib.pyplot as plt;

In [4]:
# Load dataset
from sklearn.datasets import load_iris;
from sklearn.model_selection import train_test_split

In [5]:
iris = load_iris();
X = pd.DataFrame(iris.data, columns=iris.feature_names)
print(X)

y = pd.Series(iris.target, name="target")
print(y)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  5.1               3.5                1.4               0.2
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
..                 ...               ...                ...               ...
145                6.7               3.0                5.2               2.3
146                6.3               2.5                5.0               1.9
147                6.5               3.0                5.2               2.0
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[150 rows x 4 columns]
0      0
1      0
2      0
3      0
4   

In [6]:
# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 10)


In [7]:
## DecisionTree Classifier with Pruning
'''
gini: impurity score (how mixed the classes are).
num_samples: number of training samples reaching this node.
num_samples_per_class: count of each class at this node.
predicted_class: majority class label (used if this node is a leaf).
feature_index: which feature was chosen for splitting.
threshold: the cut-off value for the split.
left / right: child nodes (subtrees).
'''
# For each node in a tree
class DecisionTreeNode:
    def __init__(self, gini, num_samples, num_sample_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_sample_per_class
        self.predicted_class = predicted_class
        self.feature_index = None
        self.threshold = None
        self.left = None
        self.right = None

In [18]:
class DecisionTreeClassification:
    '''
    max_depth: maximum depth of the tree (prevents infinite growth).
    min_samples_split: minimum samples required to allow a split.
    min_impurity_decrease: only split if Gini impurity decreases at least this much.
    '''
    def __init__(self, max_depth = None, min_samples_split = 2, min_impurity_decrease=1e-7):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_impurity_decrease = min_impurity_decrease
        
    '''
    (np.sum(y==c)/m) ** 2 for c in np.unique(y)) -> for calculating probability
    (np.sum(y==c)/m) finding number of features belonging to that particular class and divide by total to find prob.
    Sqauring each prob. and summing it up
    '''  
    def _gini(self, y):
        m = len(y)
        if m==0:
            return 0
        return 1.0- sum((np.sum(y==c)/m) ** 2 for c in np.unique(y))
    
    def _best_split(self, X, y):
        m,n = X.shape
        
        #if few samples -> no further split
        if m < self.min_samples_split:
            return None, None, 0
        
        # counting samples per class at current node
        num_parent = [np.sum(y==c) for c in np.unique(y)]
        
        #computing parent gini
        best_gini = self._gini(y) 
        
        best_idx, best_thr = None, None
        best_impurity_decrease = 0
        
        #for each feature
        '''
        Step 1: zip(X[:, idx], y)
            X[:, idx] → all values of feature idx.
            y → all class labels.
            zip pairs each feature value with its corresponding label.
            
            list(zip(X[:, idx], y))
            → [(5.1, 0), (4.9, 0), (6.2, 1)]
            
        Step 2: sorted(zip(X[:, idx], y))
            Sorts these pairs by the feature value (first element of each tuple).
            This ensures we're evaluating potential split thresholds in ascending order of the feature values.
            Example:
            sorted([(5.1, 0), (4.9, 0), (6.2, 1)])
            → [(4.9, 0), (5.1, 0), (6.2, 1)]

        Step 3: zip(*sorted(...))
            The * unpacks the sorted list of tuples into two separate sequences.
            zip(*) then groups the first elements together and the second elements together.
            👉 Example:
            zip(*[(4.9, 0), (5.1, 0), (6.2, 1)])
            → ( (4.9, 5.1, 6.2), (0, 0, 1) )
        '''
        for idx in range(n):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            # class counts on left side 
            num_left = [0]*len(np.unique(y))
            num_right = num_parent.copy()
            
            for i in range(1, m):
                c = classes[i-1]
                class_idx = list(np.unique(y)).index(c)
                num_left[class_idx] += 1
                num_right[class_idx] -= 1
                
                #Computing gini for left index
                gini_left = 1.0 - sum((num_left[x]/i)**2 for x in range(len(np.unique(y))))
                gini_right = 1.0 - sum((num_right[x]/(m-i)) ** 2 for x in range(len(np.unique(y))))
                
                gini = (i*gini_left + (m-i)*gini_right)/m
                
                impurity_decrease = best_gini - gini
                
                '''
                If two consecutive sorted feature values are equal, there is no meaningful threshold
                between them (would produce identical left/right), so skip this candidate.
                '''
                
                if(thresholds[i] == thresholds[i-1]):
                    continue
                
                '''
                If this candidate split gives a larger impurity decrease than any previous candidate
                for any feature, update the best feature best_idx, the chosen threshold best_thr (midpoint),
                and the best_impurity_decrease.
                '''
                if impurity_decrease > best_impurity_decrease:
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i-1])/2
                    best_impurity_decrease = impurity_decrease
                    
                '''
                If the best gain is smaller than min_impurity_decrease (another pre-pruning rule), we decline
                to split (return None) — otherwise return the chosen feature index, threshold, and the impurity gain.
                '''
        if best_impurity_decrease < self.min_impurity_decrease:
            return None, None, 0
        
        return best_idx, best_thr, best_impurity_decrease
            
    def _grow_tree(self, X, y, depth=0):
        # Count samples per class
        num_samples_per_class = [np.sum(y==i) for i in np.unique(y)]
        # Choose majority class at this node
        predicted_class = np.unique(y)[np.argmax(num_samples_per_class)]
        
        # Creating tree node
        node = DecisionTreeNode(
            gini = self._gini(y),
            num_samples = len(y),
            num_sample_per_class = num_samples_per_class,
            predicted_class = predicted_class,
        )
        
        '''
        This is the core recursive splitting step of the tree. If the node is allowed to
        split (depth check), it asks _best_split for the best feature and threshold; if a valid 
        split is found it partitions the data into left/right subsets, attaches the feature/threshold
        to the current node, and recursively grows left and right child nodes (incrementing depth).
        If no valid split is found, the node remains a leaf.
        '''
        
        if depth < self.max_depth:
            idx, thr, impurity = self._best_split(X, y)
            if idx is not None:
                #boolean mask that selects rows 
                indices_left = X[:, idx] < thr
                
                X_left, y_left = np.array(X[indices_left]), np.array(y[indices_left])
                X_right, y_right = np.array(X[~indices_left]), np.array(y[~indices_left])

                
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth+1)
                node.right = self._grow_tree(X_right, y_right, depth+1)
        
        return node
    
    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.n_classes = len(set(y))
        self.n_features = X.shape[1]
        self.tree_ = self._grow_tree(X, y)
        
    def _predict(self, inputs):
        #Root node of trained decision tree
        node = self.tree_
        
        while node.left:
            '''
            The input sample inputs is an array of feature values for a single data point.
            This line checks: does this sample's value for the chosen feature fall on the left
            side or the right side of the split
            '''
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class
    
    def predict(self, X):
        return [self._predict(inputs) for inputs in X]
        




In [19]:
tree_pruned = DecisionTreeClassification(max_depth = 5, min_samples_split = 5, min_impurity_decrease = 1e-3)
tree_pruned.fit(X_train.values, y_train.values)


In [20]:
y_pred = tree_pruned.predict(X_test.values)
accuracy = np.mean(y_pred == y_test.values)
print(accuracy)

0.9111111111111111


## Decision Tree Built successfully